[简书:利用Python进行数据分析](https://www.jianshu.com/p/04d180d90a3f)

In [2]:
import pandas as pd
import numpy as np

### 数据聚合与分组运算


In [3]:
df = pd.DataFrame({
    "key1" : ['a', 'a', 'b', 'b', 'a'],
    "key2" : ['one', 'two', 'one', 'two', 'one'],
    "value1" : [0, 1, 2, 3, 4],
    "value2" : [2, 6, 8, 10, 12]
})
df

,key1,key2,value1,value2
0,a,one,0,2
1,a,two,1,6
2,b,one,2,8
3,b,two,3,10
4,a,one,4,12


#### GroupBy机制

- 分组键的形式，可以有多种形式且类型不必相同：
    - 列表或数组，其长度与待分组的轴一样。
    - 表示DataFrame某个列名的值。
    - 字典或Series，给出待分组轴上的值与分组名之间的对应关系。
    - 函数，用于处理轴索引或索引中的各个标签。


- 注意事项
    - 任何分组关键词中的缺失值，都会被从结果中除去。

- **按照key1进行数据分组并计算value1列的和**

In [4]:
# GroupBy对象，无数据计算，只是包含一些有关分组键的中间数据
df_grouped = df["value1"].groupby(df["key1"])
df_grouped

In [5]:
df_grouped.sum()

key1
a    5
b    5
Name: value1, dtype: int64

- **对两个关键字进行层次化索引**

In [5]:
df_multiGrouped = df["value1"].groupby([df["key1"], df["key2"]])
df_multiGrouped.sum()

key1  key2
a     one     4
      two     1
b     one     2
      two     3
Name: value1, dtype: int64

In [6]:
df_multiGrouped.sum().unstack()

key2,one,two
key1,,
a,4,1
b,2,3


- **将列名(可以是字符串、数字或Python对象)用作分组键**

In [7]:
df_multiGrouped2 = df.groupby(["key1", "key2"])
df_multiGrouped2.sum().unstack()

value1     value2    
key2    one two    one two
key1                      
a         4   1     14   6
b         2   3      8  10

In [8]:
df_multiGrouped2.size().unstack()

key2,one,two
key1,,
a,2,1
b,1,1



##### 对分组进行迭代


In [9]:
df

,key1,key2,value1,value2
0,a,one,0,2
1,a,two,1,6
2,b,one,2,8
3,b,two,3,10
4,a,one,4,12


- **单键情况下，分组迭代**

In [10]:
for name, group in df.groupby("key1"):
    print(name)
    print("*"*60)
    print(group)

a
************************************************************
  key1 key2  value1  value2
0    a  one       0       2
1    a  two       1       6
4    a  one       4      12
b
************************************************************
  key1 key2  value1  value2
2    b  one       2       8
3    b  two       3      10


- **多重键情况下，分组迭代**。此时，元组的第一个元素将会是由键值组成的元组

In [11]:
for name, group in df.groupby(["key1", "key2"]):
    print(name)
    print("*"*60)
    print(group)

('a', 'one')
************************************************************
  key1 key2  value1  value2
0    a  one       0       2
4    a  one       4      12
('a', 'two')
************************************************************
  key1 key2  value1  value2
1    a  two       1       6
('b', 'one')
************************************************************
  key1 key2  value1  value2
2    b  one       2       8
('b', 'two')
************************************************************
  key1 key2  value1  value2
3    b  two       3      10


- **可以将分组的数据片段做成字典进行访问**

In [12]:
pieces = dict(list(df.groupby("key1")))
pieces

{'a':   key1 key2  value1  value2
 0    a  one       0       2
 1    a  two       1       6
 4    a  one       4      12, 'b':   key1 key2  value1  value2
 2    b  one       2       8
 3    b  two       3      10}

In [13]:
pieces["a"]

,key1,key2,value1,value2
0,a,one,0,2
1,a,two,1,6
4,a,one,4,12


 - **groupby默认在axis=0上分组，通过设置也可以在其他任何轴上进行分组。**

In [14]:
# 根据dtype对列进行分组
for dtype, group in df.groupby(df.dtypes, axis=1):
    print(dtype)
    print("*"*60)
    print(group)

int64
************************************************************
   value1  value2
0       0       2
1       1       6
2       2       8
3       3      10
4       4      12
object
************************************************************
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


##### 选取一列或列的子集

对于由DataFrame产生的GroupBy对象，如果用一个（单个字符串）或一组（字符串数组）列名对其进行索引，就能实现选取部分列进行聚合的目的。

In [7]:
df

,key1,key2,value1,value2
0,a,one,0,2
1,a,two,1,6
2,b,one,2,8
3,b,two,3,10
4,a,one,4,12


In [8]:
df.groupby(["key1", "key2"])[["value2"]].mean()

value2
key1 key2        
a    one        7
     two        6
b    one        8
     two       10

In [14]:
type(df.groupby(["key1", "key2"])[["value2"]].mean())

pandas.core.frame.DataFrame

##### 通过字典或Series进行分组

In [19]:
mapping = {
    "key1" : "orange",
    "key2" : "blue",
    "value1" : "red",
    "value2" : "red"
}
df.groupby(mapping, axis=1).sum()

,blue,orange,red
0,one,a,2
1,two,a,7
2,one,b,10
3,two,b,13
4,one,a,16


In [20]:
df.groupby(pd.Series(mapping), axis=1).count()

,blue,orange,red
0,1,1,2
1,1,1,2
2,1,1,2
3,1,1,2
4,1,1,2


##### 通过函数进行分组

任何被当做分组键的函数都会在各个索引值上被调用一次，其返回值就会被用作分组名称。

In [29]:
df = pd.DataFrame({
    "key1" : ['a', 'a', 'b', 'b', 'a'],
    "key2" : ['one', 'two', 'one', 'two', 'one'],
    "value1" : [0, 1, 2, 3, 4],
    "value2" : [2, 6, 8, 10, 12]
}, index = ["First", "Second", "Third", "Fourth", "Fifth"])
df

,key1,key2,value1,value2
First,a,one,0,2
Second,a,two,1,6
Third,b,one,2,8
Fourth,b,two,3,10
Fifth,a,one,4,12


In [30]:
df.groupby(len).sum()

,value1,value2
5,6,22
6,4,16


In [32]:
key_list = ['one', 'one', 'one', 'two', 'two']
df.groupby([len, key_list]).sum()

value1  value2
5 one       2      10
  two       4      12
6 one       1       6
  two       3      10

##### 根据索引级别分组

层次化索引数据集最方便的地方是它可以根据索引的一个级别进行聚合。

In [34]:
columns = pd.MultiIndex.from_arrays([ \
    ["US", "US", "CH", "CH"],[1, 5, 1, 3]], \
    names=["city", "tenor"])

df = pd.DataFrame([
    ["a", "one", 0, 2],
    ["a", "two", 1, 6],
    ["b", "two", 2, 8],
    ["b", "two", 3, 10],
    ["a", "one", 4, 12]
], columns = columns)
df

city  US      CH    
tenor  1    5  1   3
0      a  one  0   2
1      a  two  1   6
2      b  two  2   8
3      b  two  3  10
4      a  one  4  12

- 根据level关键字传递级别序号或名字，实现根据界别分组

In [38]:
df.groupby(level="city", axis = 1).sum()

city,CH,US
0,2,aone
1,7,atwo
2,10,btwo
3,13,btwo
4,16,aone


#### 数据聚合

聚合指的是任何能够从数组产生标量值的数据转换过程，比如
- count:分组中非NAN值的数量
- sum:非NAN值的平均值
- mean:非NAN值的平均数
- median:非NAN值的算数中位数
- std、var:无偏(分母为n-1)标准差和方差
- min、max:非NAN值的最大值和最小值
- prod:非NAN值的积
- first、last:第一个和最后一个非NAN的值

In [41]:
df = pd.DataFrame({
    "key1" : ['a', 'a', 'b', 'b', 'a'],
    "key2" : ['one', 'two', 'one', 'two', 'one'],
    "value1" : [0, 1, 2, 3, 4],
    "value2" : [2, 6, 8, 10, 12]
})
df

,key1,key2,value1,value2
0,a,one,0,2
1,a,two,1,6
2,b,one,2,8
3,b,two,3,10
4,a,one,4,12


- **quantile可以计算Series或DataFrame列的样本分位数。**

In [47]:
df.groupby("key1")["value1"].quantile(0.9)

key1
a    3.4
b    2.9
Name: value1, dtype: float64

In [45]:
df.groupby("key1")["value1"].median()

key1
a    1.0
b    2.5
Name: value1, dtype: float64

- **将自定义聚合函数传入aggregate或agg方法实现自定义聚合操作**

自定义聚合函数要比sum等经过优化的函数慢得多。这是因为在构造中间分组数据块时存在非常大的开销（函数调用、数据重排等）。

In [48]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
df.groupby("key1")["value1"].agg(peak_to_peak)

key1
a    4
b    1
Name: value1, dtype: int64

- **某些方法也可以用于GroupBy变量**

In [49]:
df.groupby("key1")["value1"].describe()

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,3.0,1.666667,2.081666,0.0,0.50,1.0,2.50,4.0
b,2.0,2.500000,0.707107,2.0,2.25,2.5,2.75,3.0


#### 面向列的多函数应用

In [50]:
df

,key1,key2,value1,value2
0,a,one,0,2
1,a,two,1,6
2,b,one,2,8
3,b,two,3,10
4,a,one,4,12


In [52]:
df["percent"] = df["value1"] / df["value2"]
df

,key1,key2,value1,value2,percent
0,a,one,0,2,0.000000
1,a,two,1,6,0.166667
2,b,one,2,8,0.250000
3,b,two,3,10,0.300000
4,a,one,4,12,0.333333


- **对不同的列使用不同的聚合函数**

In [54]:
df.groupby(["key1", "key2"]).agg(["mean", "sum"])

value1     value2       percent          
            mean sum   mean sum      mean       sum
key1 key2                                          
a    one       2   4      7  14  0.166667  0.333333
     two       1   1      6   6  0.166667  0.166667
b    one       2   2      8   8  0.250000  0.250000
     two       3   3     10  10  0.300000  0.300000

In [58]:
df.groupby(["key1", "key2"]).agg([("key1","mean"), ("key2", "sum")])

value1      value2        percent          
            key1 key2   key1 key2      key1      key2
key1 key2                                            
a    one       2    4      7   14  0.166667  0.333333
     two       1    1      6    6  0.166667  0.166667
b    one       2    2      8    8  0.250000  0.250000
     two       3    3     10   10  0.300000  0.300000